In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import dataneeds as need

In [2]:
%run dataneeds/tests/graph.py

Sep(',', 2) >> Cons(Node().id:Number, Node().label:String, Sep(',', 0)>>Each) []
Cons(Node().id:Number, Node().label:String, Sep(',', 0)>>Each) >> Node().id: Number
Cons(Node().id:Number, Node().label:String, Sep(',', 0)>>Each) >> Node().label: String
Cons(Node().id:Number, Node().label:String, Sep(',', 0)>>Each) >> <dataneeds.formats.Sep object at 0x7f150c3b1470>
Sep(',', 0) >> Cons(Edge().id:Number, Edge().weight:Number, Edge.source->Node().id, Edge.target->Node().id) []
Cons(Edge().id:Number, Edge().weight:Number, Edge.source->Node().id, Edge.target->Node().id) >> Edge().id: Number
Cons(Edge().id:Number, Edge().weight:Number, Edge.source->Node().id, Edge.target->Node().id) >> Edge().weight: Number
Cons(Edge().id:Number, Edge().weight:Number, Edge.source->Node().id, Edge.target->Node().id) >> Edge.source->Node().id
Cons(Edge().id:Number, Edge().weight:Number, Edge.source->Node().id, Edge.target->Node().id) >> Edge.target->Node().id
Sep(':', 0) >> Cons(Both, Edge().weight:Number, Edge

In [3]:
Node.edges.bindings

In [4]:
with need.request(Node()) as N:
    N.id, N.label#, N.edges.id
rs = N.resolve()
rs

[(dataneeds/tests/*.nlf >> Sep(',', 2) >> Cons >> Node().id:Number,
  dataneeds/tests/*.nlf >> Sep(',', 2) >> Cons >> Node().label:String),
 (dataneeds/tests/*.nef >> Sep(',', 2) >> Cons >> Both >> Node().id:Number,
  dataneeds/tests/*.nef >> Sep(',', 2) >> Cons >> Node().label:String)]

In [5]:
a,b = rs
a

(dataneeds/tests/*.nlf >> Sep(',', 2) >> Cons >> Node().id:Number,
 dataneeds/tests/*.nlf >> Sep(',', 2) >> Cons >> Node().label:String)

In [6]:
a[0].input

Cons(Node().id:Number, Node().label:String, Sep(',', 0)>>Each)

In [7]:
{'files': {'sep': {'cons': {'Node.label': {}, 'Node.id': {}}}}}

{'files': {'sep': {'cons': {'Node.id': {}, 'Node.label': {}}}}}

In [8]:
with need.request(Edge()) as E:
    E.id, E.weight, E.source.label
rs = E.resolve()
r, = rs
r

ValueError: too many values to unpack (expected 1)

In [9]:
from dataneeds.engine.dask_bag import DaskBagEngine

In [10]:
bag = DaskBagEngine().resolve(a)
bag

Node().id: Number 0 Cons(Node().id:Number, Node().label:String, Sep(',', 0)>>Each) 3
Node().label: String 1 Cons(Node().id:Number, Node().label:String, Sep(',', 0)>>Each) 3


In [11]:
from dask.async import get_sync

In [12]:
bag.compute(get=get_sync)

[(0, 'A'), (1, 'B'), (2, 'C')]

In [13]:
bag.compute(get=get_sync)

[(0, 'A'), (1, 'B'), (2, 'C')]

In [14]:
str.split?

```
\. tests/*.nef
   >> Sep(',')
      >> Cons
         >> Node.label
         >> Each
            >> Sep('.')
               >> Cons
                  >> Edge.id
                  >> Edge.weight 
|> from_files
   |> map(str.sep, sep=',')
      |> l= pluck(1)                
      |> pluck(2)
                         # explode
                         db.zip(l, _.map(len)).map(lambda x: [x[0]]*x[1]) => shape it
         |>                         .concat()
            |> map(str.sep, sep='.')
               |>                            .pluck(1)
               |>                            .pluck(3)
```

In [11]:
import dask.bag as db

In [99]:
files = db.from_filenames('dataneeds/tests/*.nef').map(str.strip)
sep = files.map(lambda x: x.split(',', 2))

labels = sep.pluck(1)

rest = sep.pluck(2)
lst = rest.map(lambda x: x.split(',') if x else [])
weights = lst.concat().map(lambda ls: ls.split(':')).pluck(1)
weights.compute()

explode = db.zip(lst.map(len), labels).map(lambda a,b: a * [b]).concat()
rq = db.zip(explode, weights)

In [100]:
rq.compute()

[('A', '.3'), ('A', '.2'), ('B', '.2'), ('B', '1.'), ('B', '.4')]

In [101]:
lst.compute()

[['0:.3:1', '1:.2:2'], ['2:.2:0', '3:1.:1', '4:.4:2'], []]